# FLB Strategy Backtesting

This notebook demonstrates how to backtest the Favorite-Longshot Bias (FLB) strategy using historical Kalshi market data.

## What We'll Do:
1. Load historical market data
2. Apply FLB strategy rules
3. Calculate returns
4. Analyze performance metrics
5. Optimize thresholds

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries loaded successfully!")

## Step 1: Load Historical Data

You'll need to export historical market data from Kalshi. The data should include:
- `ticker`: Market identifier
- `close_price`: Final price before resolution (in cents)
- `resolution`: Actual outcome ('yes' or 'no')
- `volume`: Total contracts traded
- `close_time`: When the market closed

In [ ]:
# Load your historical data
# df = pd.read_csv('../data/historical_markets.csv')

# For demonstration, create sample data
np.random.seed(42)
n_markets = 1000

# Generate realistic price distribution
prices = np.concatenate([
    np.random.beta(2, 2, n_markets//2) * 100,  # Middle range
    np.random.beta(8, 2, n_markets//4) * 100,  # Favorites
    np.random.beta(2, 8, n_markets//4) * 100   # Longshots
])

# Generate outcomes with bias (favorites win more, longshots win less)
def generate_outcome_with_bias(price):
    prob = price / 100
    if price >= 90:  # Favorites - add 3% to win probability
        prob = min(0.99, prob + 0.03)
    elif price <= 10:  # Longshots - subtract 3% from win probability
        prob = max(0.01, prob - 0.03)
    return 'yes' if np.random.random() < prob else 'no'

df = pd.DataFrame({
    'ticker': [f'MARKET-{i:04d}' for i in range(n_markets)],
    'close_price': prices.astype(int),
    'resolution': [generate_outcome_with_bias(p) for p in prices],
    'volume': np.random.randint(100, 10000, n_markets),
    'close_time': pd.date_range('2024-01-01', periods=n_markets, freq='6H')
})

print(f"Loaded {len(df)} historical markets")
df.head()

## Step 2: Define Strategy Logic

In [ ]:
class FLBBacktester:
    """Backtester for FLB strategy"""
    
    def __init__(self, favorite_threshold=90, longshot_threshold=10):
        self.favorite_threshold = favorite_threshold
        self.longshot_threshold = longshot_threshold
    
    def identify_trades(self, df):
        """Identify which markets would have been traded"""
        df = df.copy()
        
        # Identify favorites (buy YES)
        df['is_favorite'] = df['close_price'] >= self.favorite_threshold
        df['trade_favorite'] = df['is_favorite']
        df['favorite_position'] = 'yes'
        
        # Identify longshots (buy NO)
        df['is_longshot'] = df['close_price'] <= self.longshot_threshold
        df['trade_longshot'] = df['is_longshot']
        df['longshot_position'] = 'no'
        
        # Mark trades
        df['traded'] = df['trade_favorite'] | df['trade_longshot']
        
        return df
    
    def calculate_returns(self, df):
        """Calculate profit/loss for each trade"""
        df = df.copy()
        
        # For favorites (bought YES)
        df.loc[df['trade_favorite'], 'entry_price'] = df['close_price']
        df.loc[df['trade_favorite'], 'position'] = 'yes'
        df.loc[df['trade_favorite'] & (df['resolution'] == 'yes'), 'pnl'] = 100 - df['close_price']
        df.loc[df['trade_favorite'] & (df['resolution'] == 'no'), 'pnl'] = -df['close_price']
        
        # For longshots (bought NO)
        df.loc[df['trade_longshot'], 'entry_price'] = 100 - df['close_price']
        df.loc[df['trade_longshot'], 'position'] = 'no'
        df.loc[df['trade_longshot'] & (df['resolution'] == 'no'), 'pnl'] = df['close_price']
        df.loc[df['trade_longshot'] & (df['resolution'] == 'yes'), 'pnl'] = -(100 - df['close_price'])
        
        # Calculate ROI
        df['roi'] = df['pnl'] / df['entry_price'] * 100
        
        return df
    
    def run_backtest(self, df):
        """Run complete backtest"""
        df = self.identify_trades(df)
        df = self.calculate_returns(df)
        return df

# Initialize backtester
backtester = FLBBacktester(favorite_threshold=90, longshot_threshold=10)
print("Backtester initialized")

## Step 3: Run Backtest

In [ ]:
# Run backtest
results = backtester.run_backtest(df)

# Filter to traded markets only
trades = results[results['traded']].copy()

print(f"Total markets: {len(df)}")
print(f"Traded markets: {len(trades)} ({len(trades)/len(df)*100:.1f}%)")
print(f"Favorite trades: {trades['trade_favorite'].sum()}")
print(f"Longshot trades: {trades['trade_longshot'].sum()}")

## Step 4: Analyze Performance

In [ ]:
# Overall performance metrics
total_trades = len(trades)
winning_trades = len(trades[trades['pnl'] > 0])
losing_trades = len(trades[trades['pnl'] < 0])
win_rate = winning_trades / total_trades * 100

total_pnl = trades['pnl'].sum()
avg_pnl = trades['pnl'].mean()
avg_roi = trades['roi'].mean()

avg_win = trades[trades['pnl'] > 0]['pnl'].mean()
avg_loss = trades[trades['pnl'] < 0]['pnl'].mean()

print("=" * 50)
print("OVERALL PERFORMANCE")
print("=" * 50)
print(f"Total Trades: {total_trades}")
print(f"Winning Trades: {winning_trades}")
print(f"Losing Trades: {losing_trades}")
print(f"Win Rate: {win_rate:.2f}%")
print()
print(f"Total P&L: ${total_pnl:.2f}")
print(f"Average P&L per Trade: ${avg_pnl:.2f}")
print(f"Average ROI: {avg_roi:.2f}%")
print()
print(f"Average Win: ${avg_win:.2f}")
print(f"Average Loss: ${avg_loss:.2f}")
print(f"Win/Loss Ratio: {abs(avg_win/avg_loss):.2f}")

In [ ]:
# Performance by strategy
print("\n" + "=" * 50)
print("PERFORMANCE BY STRATEGY")
print("=" * 50)

for strategy, label in [('trade_favorite', 'FAVORITES'), ('trade_longshot', 'LONGSHOTS')]:
    strat_trades = trades[trades[strategy]]
    if len(strat_trades) == 0:
        continue
    
    strat_wins = len(strat_trades[strat_trades['pnl'] > 0])
    strat_win_rate = strat_wins / len(strat_trades) * 100
    strat_total_pnl = strat_trades['pnl'].sum()
    strat_avg_pnl = strat_trades['pnl'].mean()
    strat_avg_roi = strat_trades['roi'].mean()
    
    print(f"\n{label}:")
    print(f"  Trades: {len(strat_trades)}")
    print(f"  Win Rate: {strat_win_rate:.2f}%")
    print(f"  Total P&L: ${strat_total_pnl:.2f}")
    print(f"  Avg P&L: ${strat_avg_pnl:.2f}")
    print(f"  Avg ROI: {strat_avg_roi:.2f}%")

## Step 5: Visualizations

In [ ]:
# Cumulative P&L over time
trades_sorted = trades.sort_values('close_time')
trades_sorted['cumulative_pnl'] = trades_sorted['pnl'].cumsum()

plt.figure(figsize=(12, 6))
plt.plot(trades_sorted['close_time'], trades_sorted['cumulative_pnl'], linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.title('Cumulative P&L Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative P&L ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final cumulative P&L: ${trades_sorted['cumulative_pnl'].iloc[-1]:.2f}")

In [ ]:
# P&L distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(trades['pnl'], bins=50, edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Break-even')
ax1.axvline(x=trades['pnl'].mean(), color='g', linestyle='--', linewidth=2, label=f'Mean: ${trades["pnl"].mean():.2f}')
ax1.set_title('P&L Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('P&L ($)')
ax1.set_ylabel('Frequency')
ax1.legend()
ax1.grid(True, alpha=0.3)

# ROI distribution
ax2.hist(trades['roi'], bins=50, edgecolor='black', alpha=0.7, color='orange')
ax2.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax2.axvline(x=trades['roi'].mean(), color='g', linestyle='--', linewidth=2, label=f'Mean: {trades["roi"].mean():.2f}%')
ax2.set_title('ROI Distribution', fontsize=14, fontweight='bold')
ax2.set_xlabel('ROI (%)')
ax2.set_ylabel('Frequency')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Win rate by price bucket
# Divide price range into buckets
price_buckets = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
trades['price_bucket'] = pd.cut(trades['close_price'], bins=price_buckets)

bucket_stats = trades.groupby('price_bucket').agg({
    'pnl': ['count', 'mean', lambda x: (x > 0).sum() / len(x) * 100]
}).round(2)

bucket_stats.columns = ['Count', 'Avg P&L', 'Win Rate %']
print("\nPerformance by Price Bucket:")
print(bucket_stats)

## Step 6: Threshold Optimization

In [ ]:
# Test different threshold combinations
favorite_thresholds = range(85, 98, 2)
longshot_thresholds = range(2, 16, 2)

optimization_results = []

for fav_thresh in favorite_thresholds:
    for long_thresh in longshot_thresholds:
        bt = FLBBacktester(favorite_threshold=fav_thresh, longshot_threshold=long_thresh)
        test_results = bt.run_backtest(df)
        test_trades = test_results[test_results['traded']]
        
        if len(test_trades) > 0:
            optimization_results.append({
                'favorite_threshold': fav_thresh,
                'longshot_threshold': long_thresh,
                'num_trades': len(test_trades),
                'win_rate': (test_trades['pnl'] > 0).sum() / len(test_trades) * 100,
                'total_pnl': test_trades['pnl'].sum(),
                'avg_pnl': test_trades['pnl'].mean(),
                'avg_roi': test_trades['roi'].mean()
            })

opt_df = pd.DataFrame(optimization_results)

# Find best configuration by total P&L
best_config = opt_df.loc[opt_df['total_pnl'].idxmax()]
print("\nBest Configuration (by Total P&L):")
print(f"Favorite Threshold: {best_config['favorite_threshold']:.0f}")
print(f"Longshot Threshold: {best_config['longshot_threshold']:.0f}")
print(f"Number of Trades: {best_config['num_trades']:.0f}")
print(f"Win Rate: {best_config['win_rate']:.2f}%")
print(f"Total P&L: ${best_config['total_pnl']:.2f}")
print(f"Average ROI: {best_config['avg_roi']:.2f}%")

In [ ]:
# Heatmap of total P&L by threshold combinations
pivot_pnl = opt_df.pivot(index='favorite_threshold', columns='longshot_threshold', values='total_pnl')

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_pnl, annot=True, fmt='.0f', cmap='RdYlGn', center=0)
plt.title('Total P&L by Threshold Combinations', fontsize=16, fontweight='bold')
plt.xlabel('Longshot Threshold')
plt.ylabel('Favorite Threshold')
plt.tight_layout()
plt.show()

## Step 7: Risk Metrics

In [ ]:
# Calculate drawdown
trades_sorted['max_cumulative'] = trades_sorted['cumulative_pnl'].cummax()
trades_sorted['drawdown'] = trades_sorted['cumulative_pnl'] - trades_sorted['max_cumulative']
max_drawdown = trades_sorted['drawdown'].min()

# Sharpe ratio (simplified - assuming daily returns)
returns = trades['pnl']
sharpe = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0

print("=" * 50)
print("RISK METRICS")
print("=" * 50)
print(f"Maximum Drawdown: ${max_drawdown:.2f}")
print(f"Sharpe Ratio (annualized): {sharpe:.2f}")
print(f"Volatility (std dev): ${returns.std():.2f}")
print(f"Best Trade: ${trades['pnl'].max():.2f}")
print(f"Worst Trade: ${trades['pnl'].min():.2f}")

## Conclusions

Based on this backtest:

1. **Strategy Viability**: The FLB strategy shows [positive/negative] expected value
2. **Optimal Thresholds**: Best performance with favorite={best_favorite}¢, longshot={best_longshot}¢
3. **Win Rate**: Achieved {win_rate}% win rate
4. **Risk/Reward**: Average ROI of {avg_roi}% with max drawdown of ${max_drawdown}

## Next Steps

1. Test on different time periods for robustness
2. Implement transaction costs and slippage
3. Add position sizing optimization
4. Consider market-specific filters
5. Develop Strategy B (Alpha Specialist) for additional edge